In [2]:
import findspark
findspark.init('/home/sushant/spark-2.1.0-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('MLspark').getOrCreate()

In [6]:
from pyspark.ml.regression import LinearRegression

In [7]:
!pwd

/home/sushant/Documents/SparkUdemyCourse


In [8]:
training = spark.read.format('libsvm').load('./Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/sample_linear_regression_data.txt')

In [10]:
training.show()
# note that we have a label column and a feature column.

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

The feature column inside of it has a vector and that is a vector of all te features that belong to that row. 

In [11]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')
# These are the default values for featuresCol, labelCol, and predictionCol. We can change them if we want.

In [12]:
lrModel = lr.fit(training)

In [14]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [15]:
lrModel.intercept

0.14228558260358093

In [16]:
trainSummary = lrModel.summary

In [17]:
trainSummary.r2

0.027839179518600154

Not a great model. 

In [18]:
trainSummary.rootMeanSquaredError

10.16309157133015